In [ ]:
import numpy as np 
import pandas as pd
#METÓDO GRADIENTE 
def gradiente(A,x0,b, tol=1e-10):
    x = x0.copy()
    k = 0
    r = b - A @ x0
    su = [x]
    res = [r]
    while np.linalg.norm(x) > tol and k <= 500:
        alpha = (r @ r) / (r @ A @ r)
        x = x +  alpha * r
        r = b - A @ x
        su.append(x)
        res.append(r) 
        k = k + 1 
    #DIBUJO
    nres = [np.linalg.norm(resv) for resv in res]
    iteraciones = np.arange(len(nres))
    plt.scatter(iteraciones, nres, color='b', alpha=0.7)
    plt.title('Evolución de los residuos')
    plt.xlabel('Iteración')
    plt.ylabel('Norma del residuo')
    plt.grid(True, linestyle='--', alpha=0.3,color = 'y')
    plt.show()
    return su, res

In [ ]:
#MEETODO DEL GRADIENTE CONJUGADO
def gradiente_conjugado( A , b , x0 , tol =  1e-8 ):
    x = x0.copy()
    r = A @ x - b
    p = -r
    k = 0
    history = {
        'iteration' : [],
        'norm_x' : [],
        'norm-r' : []
    }
    while np.linalg.norm(r) > tol:
        norm_x = np.linalg.norm(x)
        norm_r =np.linalg.norm(r)
        history['iteration'].append(k)
        history['norm_x'].append(norm_x)
        history['norm-r'].append(norm_r)
        Ap = A @ p
        alpha= r @ r /(p @ Ap)
        x = x + alpha*p
        r_new = r + alpha*Ap
        beta = r_new @ r_new / (r @ r)
        p = -r_new + beta * p
        r = r_new
        k += 1
        df = pd.DataFrame(history)
    return x,k,df

In [ ]:
#METÓDO DE GAUSS-SEIDEL CON SUSTITUCION PROGRESIVA
def gauss_seidel(A,b,x0,tol):
    x=x0.copy()
    k=0
    while np.linalg.norm(x0 - np.linalg.inv(A)@b)>tol and k<=100:
        x_ant=x.copy()
        for i in range(len(A)):
            suma1=0
            suma2=0
            for j in range(0,i):
                suma1+=A[i,j]*x[j]
            for j in range(i,len(A)-1):
                suma2+=A[i,j]*x_ant[j]
            x[i]=(b[i]-suma1-suma2)/A[i,i]
            k+=1
    return x,k 

In [ ]:
A=np.array([[1,2,0],[2,1,0],[1,1,1]])
Asdp=A@A.T
b=np.array([1,2,3])
x0=np.zeros(len(b))
tol=0.05
print("VERIFICAR QUE LAMATRIZ SEA SDP:",np.linalg.eigvals(Asdp) )
gauss_seidel(Asdp,b,x0,tol) 

In [ ]:
#METÓDO DE LA POTENCIA 
def power_method(A,q,iter):
    for k in range(iter):
        z = A@ q
        q = z/np.linalg.norm(z)
        nu= q.T @ A @ q
    return nu,q

In [ ]:

def matriz_hilbert(n):
    H = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(n):
            H[i, j] = 1 / (i + j + 1)
    return H

# Crear matriz Hilbert 6x6
H6 =matriz_hilbert(6)
print(H6)

In [ ]:
#METÓDO DE LA POTENCIA INVERSA
def invpower(A, z0, mu, tol=1e-8, nmax=100):


    n = A.shape[0]
    M = A - mu * np.eye(n)
    q = z0 / np.linalg.norm(z0)
    res = tol + 1
    niter = 0

    while res > tol and niter < nmax:
        niter += 1
        z = np.linalg.solve(M, q)  # resolver (A - mu I) z = q
        q2 = z / np.linalg.norm(z)
        lam = np.dot(q2.T, A @ q2)  # cociente de Rayleigh
        res = np.linalg.norm(A @ q2 - lam * q2)
        costheta = abs(np.dot(q2.T, q))

        if costheta < 5e-2:
            print("Valor propio múltiple detectado (ángulo entre vectores casi ortogonal)")
            break

        q = q2

    sigma = lam
    err = res
    x = q2
    return sigma, x, niter, err